# CAMMAC https://cammac.readthedocs.io
S.Sénési for Météo-France - sept 2019 to march 2021

# Build a figure showing changes for one (raw or derived) variable, three SSPs and three horizons


## Parameters stand in first cell, are either commented here or in the doc (see above)

### A few commands below are specific to the Notebook environment, and can be safely commented out

## Default settings (some may be overriden by Papermill - this would show in next cell in the execution output notebook)

In [ ]:
import os
do_test                = False

figure_name            = "FigTS-2-9"
version                = ""     #Used as suffix in automatic figure name
scheme                 = "AR5"
confidence_factor    = 1.645  # For AR6 comprehensive scheme : Multiplicative factor applied to control run 
                              # variability for deciding a change is significant (besides sqrt(2))
sign_threshold       = 0.66   # For AR6 simple scheme : threshold on cross-model change sign agreeement fraction
same_models_for_var    = False

data_versions_tag      = "20200719d"
data_versions_dir      = os.getenv("CAMMAC")+"/data"
excluded_models        = []
included_models        = None

variable               = "pr"
#threshold              = 0.1/(24*3600) # 0.1 mm/day, in SI
threshold              = None
table                  = "Amon"
derivation             = "plain"
season                 = "ANN"
field_type             = "means_rchange"

experiments            = ["ssp126","ssp245","ssp585"]
horizons               = ["2021-2040","2041-2060","2081-2100"]
ref_period             = "1995-2014"
ref_experiment         = "historical"

manual_title           = None
auto_title_end         = "for three SSPs and near to long-term"
plot_args              = {"colors":"-40 -20 -10  -5 -2  0 2  5 10 20 40 "}
#base_plot_args_50={"colors":"-50 -30 -20 -10 -5  0 5 10 20 30 50 "}

variability_models     = None
variability_excluded_models = []
variab_sampling_args   = {"house_keeping":True,"compute":True,"detrend":True,"shift":100,"nyears":20,"number":20}

figure_details         = {"page_width":2450,"page_height":3444, "insert_width":1800,"pt":48, "ybox":133,"y":52}
use_cached_proj_fields = True
drop_old_figures       = False
common_grid            = "r360x180"
outdir                 = "./figures"

In [ ]:
do_test=False
if do_test :
    version             = "_test"
    horizons            = ["2040","2040","2040"]
    ref_period          = "2014" 
    included_models     = ["CNRM-CM6-1"]
    experiments         = ["ssp126","ssp126","ssp126"]
    variab_sampling_args= {}
    


## Load libraries

In [ ]:
import sys

from climaf.api import *
climaf.cache.stamping=False

from CAMMAClib.changes    import change_figure_with_caching
from CAMMAClib.ancillary  import create_labelbar, prettier_label
from CAMMAClib.mips_et_al import TSU_metadata, read_versions_dictionnary

# Load some user settings, if available
settings_file=os.getenv("CAMMAC_USER_PYTHON_CODE_DIR",".")+'/cammac_user_settings.py'
if os.path.exists(settings_file) :
    exec(compile(open(settings_file).read(),settings_file,'exec'))

### If using a notebook, use wide display

In [ ]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))

## The basic engine is function change_figure_with_caching, which has numerous settings, as displayed further below

## Next function allows to set most values from global variable, and the others using arguments

In [ ]:
def afigure(experiment,projection_period, title=None ,labelbar="False",outfile=None,plot_args=plot_args):
    
    global metadata
    
    fig_file,fig,dic,variab_models,models=change_figure_with_caching(
        ref_period=ref_period, proj_period=projection_period, 
        variable=variable, table=table, ref_experiment=ref_experiment,
        experiment=experiment, season=season,
        derivation_label=derivation, field_type=field_type,
        title=title, custom_plot=plot_args, labelbar=labelbar, 
        data_versions_tag=data_versions_tag, data_versions_dir=data_versions_dir,
        outdir=outdir, outfile=outfile,
        #
        common_grid=common_grid, mask=None,
        variab_sampling_args=variab_sampling_args,
        excluded_models=excluded_models, models=included_models,
        variability_models=variability_models, variability_excluded_models=variability_excluded_models,
        cache_dir="./cache", read=use_cached_proj_fields, write=True, 
        print_statistics=True , deep=None,  threshold=threshold,
        same_models_for_variability_and_changes=same_models_for_var,
        scheme=scheme,
        low_change_agree_threshold=confidence_factor,
        change_sign_agree_threshold = sign_threshold,
        drop=drop_old_figures
        )
    metadata += TSU_metadata([ref_experiment,experiment],models,       variable,table,data_versions,experiment)
    metadata += TSU_metadata(["piControl"],              variab_models,variable,table,data_versions,experiment)
    return fig

## Compute figures for the three experiments and three terms

In [ ]:
lines=[]
metadata=""
data_versions=read_versions_dictionnary(data_versions_tag, data_versions_dir)

for experiment in experiments :
    line=[]
    for horizon in horizons :
        f=afigure(experiment,horizon,title=horizon+" "+prettier_label.get(experiment,experiment))
        line.append(f)
    lines.append(line)

In [ ]:
import os.path
if not os.path.exists(outdir):
    os.makedirs(outdir)
with open("%s/%s%s_md.txt"%(outdir,figure_name,version),"w") as f: 
    f.write(metadata)

## Create the common labelbar and assemble it with plots

In [ ]:
# create a figure wih labelbar (using same plot settings as for figures above)
labfile="./fig_with_label.png"
labelbar_figure=afigure(experiments[0],horizons[0],labelbar="True",outfile=labfile)

# Call utility function for extracting labelbar and adding fill pattern for shadings
create_labelbar(labfile, "./insert.png",missing=False,scheme=scheme)

if manual_title is None : 
    title="%s %s %s %s "%(prettier_label.get(season,season),prettier_label.get(variable,variable),prettier_label.get(field_type,field_type),auto_title_end) 
else :
    title = manual_title

# Create multi-panel figure
page=cpage(lines, title=title,insert="./insert.png",**figure_details)

outfile="change_3SSPS_3horizons_%s_%s_%s%s.png"%(variable,derivation,data_versions_tag,version)
if drop_old_figures : cdrop(page)
cfile(page,outdir+"/"+outfile)
os.system("cd %s ; ln -sf %s %s%s.png"%(outdir,outfile,figure_name,version))
#
small=outfile.replace(".png",".small.png")
os.system("cd %s ; convert -geometry 50%% %s %s"%(outdir,outfile,small))
os.system("cd %s ; ln -sf %s %s%s_small.png"%(outdir,small,figure_name,version))
#
os.system("convert -geometry 50%% %s %s"%(outfile,outfile.replace(".png",".small.png")))
os.system("rm ./insert.png")
os.system("rm %s"%labfile)

## If using a notebook , display result on-line

In [ ]:
#Image(outdir+"/"+outfile,width=600)